## Importações

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Leitura

In [8]:
spark = SparkSession.builder \
    .appName("AnaliseENEM") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

leitura = spark.read.option("header", "true") \
               .option("sep", ";") \
               .option("encoding", "ISO-8859-1") \
               .option("inferSchema", "true") \
               .csv("data_raw/MICRODADOS_ENEM_2021.csv")

## Verificação da qualidade dos dados na camada bronze:

###  1. Dados faltantes por colunas

In [ ]:
colunas_nulas = leitura.select([count(when(col(c).isNull(), c)).alias(c) 
                                for c in leitura.columns
                                ]).toPandas().transpose().reset_index()

contagem_total = leitura.count()

plt.figure(figsize=(16, 6))

colunas_nulas.columns = ['Coluna', 'Qtd_Nulos']
colunas_nulas['Perc_Nulos'] = (colunas_nulas['Qtd_Nulos'] / contagem_total) * 100

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

relatorio_silver = colunas_nulas.sort_values('Perc_Nulos', ascending=False)

relatorio_silver